In [51]:
# Librerías.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy.stats import ttest_rel

In [52]:

# Configuración de estilo para los gráficos.
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
Carpeta_Exportar = 'C:/Users/Patricio/Documents/Codigo/Python/Investigacion/Tesis/Data/Bases definitivas/'

In [54]:
Bases = ['Generales', 'Ballotage']

In [55]:
Categorias_PASO = ['Left_Wing',
                   'Progressivism',
                   'Centre',
                   'Moderate_Right_A',
                   'Moderate_Right_B',
                   'Right_Wing_Libertarian']

In [56]:
Variable_Y = 'Categoria_PASO_2023'
Variable_X_Izquierda = 'CT_Item_X_Izq'
Variable_X_Derecha = 'CT_Item_X_Der'

In [57]:
Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]

In [58]:
# Crear carpeta para guardar gráficos si no existe.
Carpeta_Graficos = f'{Carpeta_Exportar}Graficos_Cleveland/'
os.makedirs(Carpeta_Graficos, exist_ok=True)

In [59]:
# Colores distintivos.
Color_Progresista = '#2E8B57'  # Verde
Color_Conservador = '#87CEEB'  # Celeste

In [60]:
def Crear_Grafico_Cleveland(df, Item_Numero, Tipo_Item, Base_Nombre):

    """
    Crea un gráfico de Cleveland para un ítem específico comparando 
    medias de cambio de Tiempo entre izquierda y derecha por categoría.
    Usa prueba de Wilcoxon para datos no normales y guarda resultados.

    """

    # Filtrar datos por categorías de interés.
    df_filtrado = df[df[Variable_Y].isin(Categorias_PASO)].copy()
    
    # Nombres de variables para este ítem.
    Variable_Izquierda = f'{Variable_X_Izquierda.replace("X", str(Item_Numero))}'
    Variable_Derecha = f'{Variable_X_Derecha.replace("X", str(Item_Numero))}'
    
    # Calcular estadísticas por categoría.
    Estadisticas = []
    for Categoria in Categorias_PASO:
        Datos_Categoria = df_filtrado[df_filtrado[Variable_Y] == Categoria]
        
        if len(Datos_Categoria) > 0:
            # Eliminar valores faltantes para análisis pareado.
            Datos_Validos = Datos_Categoria.dropna(
                subset = [Variable_Izquierda, Variable_Derecha]
            )
            
            # Mínimo 5 observaciones para prueba de Wilcoxon.
            if len(Datos_Validos) >= 5:
                Media_Izq = Datos_Validos[Variable_Izquierda].mean()
                Media_Der = Datos_Validos[Variable_Derecha].mean()
                Mediana_Izq = Datos_Validos[Variable_Izquierda].median()
                Mediana_Der = Datos_Validos[Variable_Derecha].median()
                N_Muestra = len(Datos_Validos)
                Diferencia_Media = Media_Der - Media_Izq
                Diferencia_Mediana = Mediana_Der - Mediana_Izq
                
                # Prueba de Wilcoxon para datos pareados no normales.
                from scipy.stats import wilcoxon
                try:
                    # Calcular diferencias para cada par.
                    Diferencias = (Datos_Validos[Variable_Derecha] - 
                                 Datos_Validos[Variable_Izquierda])
                    
                    # Verificar que hay diferencias no nulas.
                    if (Diferencias != 0).sum() >= 3:
                        Estadistica_W, P_Valor = wilcoxon(
                            Datos_Validos[Variable_Derecha], 
                            Datos_Validos[Variable_Izquierda],
                            alternative = 'two-sided'
                        )
                    else:
                        # No hay suficientes diferencias para el test.
                        Estadistica_W = np.nan
                        P_Valor = 1.0
                        
                except Exception as e:
                    # En caso de error, asignar valores nulos.
                    Estadistica_W = np.nan
                    P_Valor = 1.0
                
                # Determinar significancia estadística.
                if P_Valor < 0.001:
                    Asteriscos = '***'
                    Significancia = 'p < 0.001'
                elif P_Valor < 0.01:
                    Asteriscos = '**'
                    Significancia = 'p < 0.01'
                elif P_Valor < 0.05:
                    Asteriscos = '*'
                    Significancia = 'p < 0.05'
                elif P_Valor < 0.10:
                    Asteriscos = '†'
                    Significancia = 'p < 0.10'
                else:
                    Asteriscos = ''
                    Significancia = 'n.s.'
                
                Estadisticas.append({
                    'Item': Item_Numero,
                    'Tipo_Item': Tipo_Item,
                    'Base': Base_Nombre,
                    'Categoria': Categoria,
                    'N_Muestra': N_Muestra,
                    'Media_Izquierda': Media_Izq,
                    'Media_Derecha': Media_Der,
                    'Mediana_Izquierda': Mediana_Izq,
                    'Mediana_Derecha': Mediana_Der,
                    'Diferencia_Media': Diferencia_Media,
                    'Diferencia_Mediana': Diferencia_Mediana,
                    'Estadistica_Wilcoxon': Estadistica_W,
                    'P_Valor': P_Valor,
                    'Significancia': Significancia,
                    'Asteriscos': Asteriscos
                })
    
    # Guardar estadísticas en archivo Excel.
    if Estadisticas:
        df_estadisticas = pd.DataFrame(Estadisticas)
        Archivo_Stats = (f'{Carpeta_Graficos}Estadisticas_Item_'
                        f'{Item_Numero}_{Base_Nombre}_Tiempo.xlsx')
        df_estadisticas.to_excel(Archivo_Stats, index = False)
        print(f'Estadísticas guardadas: {Archivo_Stats}')
    
    # Crear el gráfico con más espacio para anotaciones.
    fig, ax = plt.subplots(figsize = (16, 8))
    
    # Determinar color según tipo de ítem.
    Color_Principal = (Color_Progresista if Tipo_Item == 'Progresista' 
                      else Color_Conservador)
    
    # Posiciones Y invertidas para orden correcto.
    Y_Posiciones = list(reversed(range(len(Estadisticas))))
    
    # Encontrar los valores extremos de los datos para ajustar límites.
    Valores_Minimos = []
    Valores_Maximos = []
    
    # Dibujar líneas conectoras y puntos.
    for i, Stats in enumerate(Estadisticas):
        Y_Pos = Y_Posiciones[i]
        
        # Actualizar valores extremos.
        Valores_Minimos.append(Stats['Media_Izquierda'])
        Valores_Minimos.append(Stats['Media_Derecha'])
        Valores_Maximos.append(Stats['Media_Izquierda'])
        Valores_Maximos.append(Stats['Media_Derecha'])
        
        # Línea conectora con grosor según significancia.
        Grosor_Linea = 3 if Stats['Asteriscos'] else 1.5
        Alpha_Linea = 0.8 if Stats['Asteriscos'] else 0.4
        
        ax.plot([Stats['Media_Izquierda'], Stats['Media_Derecha']], 
                [Y_Pos, Y_Pos], 
                color = 'gray', 
                alpha = Alpha_Linea, 
                linewidth = Grosor_Linea)
        
        # Puntos con tamaño según muestra.
        Tamano_Punto = min(50 + Stats['N_Muestra'], 200)
        
        ax.scatter(Stats['Media_Izquierda'], Y_Pos, 
                  color = Color_Principal, 
                  s = Tamano_Punto, 
                  alpha = 0.8, 
                  edgecolors = 'black',
                  linewidth = 0.5,
                  label = 'Izquierda' if i == 0 else "")
        
        ax.scatter(Stats['Media_Derecha'], Y_Pos, 
                  color = Color_Principal, 
                  s = Tamano_Punto, 
                  alpha = 0.8, 
                  marker = 'D',
                  edgecolors = 'black',
                  linewidth = 0.5,
                  label = 'Derecha' if i == 0 else "")
    
    # Calcular límites apropiados basados en los datos.
    if Valores_Minimos and Valores_Maximos:
        Valor_Min = min(Valores_Minimos)
        Valor_Max = max(Valores_Maximos)
        Rango = Valor_Max - Valor_Min
        Margen = Rango * 0.15
        
        # Establecer límites con espacio extra para anotaciones.
        Limite_Izquierdo = Valor_Min - Margen
        Limite_Derecho = Valor_Max + Margen * 3  # Más espacio a la derecha.
        
        ax.set_xlim(Limite_Izquierdo, Limite_Derecho)
        
        # Posición X para las anotaciones (fuera del área de datos).
        Posicion_X_Anotacion = Valor_Max + Margen * 0.5
    else:
        # Valores por defecto si no hay datos.
        ax.set_xlim(-12, 15)
        Posicion_X_Anotacion = 11
    
    # Agregar anotaciones fuera del área principal.
    for i, Stats in enumerate(Estadisticas):
        Y_Pos = Y_Posiciones[i]
        
        # Anotación con toda la información.
        Texto_Anotacion = (f'n={Stats["N_Muestra"]} | '
                          f'Δ={Stats["Diferencia_Media"]:.3f}'
                          f'{Stats["Asteriscos"]}')
        
        ax.text(Posicion_X_Anotacion, Y_Pos, 
                Texto_Anotacion,
                fontsize = 9, 
                va = 'center',
                alpha = 0.7)
    
    # Línea de referencia en x=0.
    ax.axvline(x = 0, color = 'black', linestyle = '--', 
               alpha = 0.5, linewidth = 1)
    
    # Configuración del eje Y.
    ax.set_ylim(-0.5, len(Estadisticas) - 0.5)
    
    # Etiquetas del eje Y.
    Nombres_Categorias = [Stats['Categoria'] for Stats in Estadisticas]
    ax.set_yticks(Y_Posiciones)
    ax.set_yticklabels(Nombres_Categorias)
    
    # Títulos y etiquetas.
    Subtipo = "Progresista" if Tipo_Item == 'Progresista' else "Conservador"
    ax.set_title(f'Ítem {Item_Numero} ({Subtipo}) - {Base_Nombre}', 
                fontsize = 16, 
                fontweight = 'bold')
    ax.set_xlabel('Medias de Cambio de Tiempo', fontsize = 12)
    ax.set_ylabel('Categorías Políticas', fontsize = 12)
    
    # Leyenda mejorada.
    ax.legend(loc = 'upper left', frameon = True, 
             fancybox = True, shadow = True)
    
    # Agregar nota sobre significancia.
    Nota_Texto = ('Significancia: *** p<0.001, ** p<0.01, * p<0.05, '
                 '† p<0.10\nPrueba: Wilcoxon signed-rank test')
    ax.text(0.98, 0.02, Nota_Texto, transform = ax.transAxes,
           fontsize = 8, ha = 'right', va = 'bottom',
           bbox = dict(boxstyle = 'round', facecolor = 'wheat', 
                      alpha = 0.3))
    
    # Mejorar apariencia.
    ax.grid(True, alpha = 0.3, linestyle = ':')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Ajustar márgenes para acomodar las anotaciones.
    plt.subplots_adjust(left = 0.15, right = 0.80, top = 0.92, 
                       bottom = 0.08)
    
    # Guardar gráfico.
    Nombre_Archivo = f'Cleveland_Item_{Item_Numero}_{Base_Nombre}_Tiempo.svg'
    plt.savefig(f'{Carpeta_Graficos}{Nombre_Archivo}', 
                dpi = 300, 
                bbox_inches = 'tight',
                pad_inches = 0.1)
    
    plt.close()
    
    print(f'Gráfico guardado: {Nombre_Archivo}')
    
    return Estadisticas

In [61]:
# Consolidar todas las estadísticas en un único archivo.
Todas_Estadisticas = []

In [ ]:
for Base in Bases:
    # Cargar base de datos.
    df = pd.read_excel(f'{Carpeta_Exportar}{Base}.xlsx')

    print(f'\nProcesando base: {Base}')
    
    # Crear gráficos para ítems progresistas.
    for Item in Items_Progresistas:
        Stats = Crear_Grafico_Cleveland(df, Item, 'Progresista', Base)
        Todas_Estadisticas.extend(Stats)
    
    # Crear gráficos para ítems conservadores.
    for Item in Items_Conservadores:
        Stats = Crear_Grafico_Cleveland(df, Item, 'Conservador', Base)
        Todas_Estadisticas.extend(Stats)

In [ ]:
# Guardar archivo consolidado con todas las estadísticas.
if Todas_Estadisticas:
    df_Consolidado = pd.DataFrame(Todas_Estadisticas)
    Archivo_Consolidado = f'{Carpeta_Graficos}Estadisticas_CT_Comparados_Candidatos.xlsx'
    
    # Guardar con formato mejorado.
    with pd.ExcelWriter(Archivo_Consolidado, engine = 'openpyxl') as Writer:
        df_Consolidado.to_excel(Writer, sheet_name = 'Estadisticas', 
                               index = False)
        
        # Crear resumen por tipo de ítem.
        Resumen = df_Consolidado.groupby(['Tipo_Item', 'Base']).agg({
            'P_Valor': ['mean', 'median', lambda x: (x < 0.05).sum()],
            'N_Muestra': ['mean', 'sum']
        }).round(4)
        
        Resumen.to_excel(Writer, sheet_name = 'Resumen')
    
    print(f'\nEstadísticas consolidadas guardadas en: {Archivo_Consolidado}')

print(f'\n¡Proceso completado! Archivos en: {Carpeta_Graficos}')